In [5]:
import pandas as pd
import numpy as np
from mca import diagsvd
# import mca
import functools
import sklearn.model_selection as ms
from sklearn.preprocessing import StandardScaler as stdc
import gc
import _pickle as pickle
categorical = ['source_system_tab','source_type','gender','city','bd']
numerical = ['song_length', 'song_year']

def standarize_data(df):
	df = stdc().fit_transform(df)
	return df



In [2]:
def split(X, y, proportion):
	print('Train \nRaw data: \n ', 'Shape: ', X.shape, 'Type: ', type(X))
	print('Target \nRaw data: \n ', 'Shape: ', y.shape)
	# I split the data between the X and the target value

	# X is a dataframe not a np.array
	print('Size of X: ', X.shape, '\nSize of y: ', y.shape)
	(X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=proportion, random_state=1, stratify=y)
	(X_test, X_val, y_test, y_val) = ms.train_test_split(X_test, y_test, test_size=.5, random_state=1, stratify=y_test)
	print('\n New train shape: ', X_train.shape, ' \n New test shape: ', X_test.shape, '\n New val shape: ',
	      X_val.shape, '\n Type: ',type(X_train))

	return X_train, X_test, X_val, y_train, y_test, y_val


# Partitioning

In [3]:
file = 'def_training.csv'
X = pd.read_csv('../Data/' + file)
X['city'] = X['city'].astype('category')
y = X['target']

X_num = X[numerical]

T = pd.read_csv('../Data/def_test.csv')
T['city'] = T['city'].astype('category')

T_num = T[numerical]


Particiono

In [6]:
DummiesX = pd.get_dummies(data=X[categorical], prefix_sep='|')
T_cat = pd.get_dummies(data=T[categorical], prefix_sep='|')

print(T_cat.shape)
print(DummiesX.shape)

(X_num_train, X_num_test, X_num_val, y_train, y_test, y_val) = split(X_num, y, 0.5)
(X_cat_train, X_cat_test, X_cat_val, y_train, y_test, y_val) = split(DummiesX, y, 0.5)
del DummiesX , X_num
gc.collect()

(2556790, 42)
(7347633, 42)
Train 
Raw data: 
  Shape:  (7347633, 2) Type:  <class 'pandas.core.frame.DataFrame'>
Target 
Raw data: 
  Shape:  (7347633,)
Size of X:  (7347633, 2) 
Size of y:  (7347633,)

 New train shape:  (3673816, 2)  
 New test shape:  (1836908, 2) 
 New val shape:  (1836909, 2) 
 Type:  <class 'pandas.core.frame.DataFrame'>
Train 
Raw data: 
  Shape:  (7347633, 42) Type:  <class 'pandas.core.frame.DataFrame'>
Target 
Raw data: 
  Shape:  (7347633,)
Size of X:  (7347633, 42) 
Size of y:  (7347633,)

 New train shape:  (3673816, 42)  
 New test shape:  (1836908, 42) 
 New val shape:  (1836909, 42) 
 Type:  <class 'pandas.core.frame.DataFrame'>


72

In [7]:
X_cat_train[X_cat_train.columns] = X_cat_train[X_cat_train.columns].astype(np.uint8)
X_cat_test[X_cat_test.columns] = X_cat_test[X_cat_test.columns].astype(np.uint8)
X_cat_val[X_cat_val.columns] = X_cat_val[X_cat_val.columns].astype(np.uint8)
T_cat[T_cat.columns] = T_cat[T_cat.columns].astype(np.uint8)
X_cat_test.head()

,gender,source_system_tab|discover,source_system_tab|explore,source_system_tab|listen with,source_system_tab|my library,source_system_tab|notification,source_system_tab|radio,source_system_tab|search,source_system_tab|settings,source_type|album,...,city|13,city|14,city|15,city|16,city|17,city|18,city|19,city|20,city|21,city|22
5296846,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5530281,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5768063,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
174007,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3260021,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Elimino las columnas que tienen cero para todas las filas de algun subconjunto

In [8]:
T_sum =  (T_cat.values.sum(axis=0) != 0)
X_train_sum = (X_cat_train.values.sum(axis=0) != 0)
X_test_sum = (X_cat_test.values.sum(axis=0) != 0)
X_val_sum = (X_cat_val.values.sum(axis=0) != 0)
index =np.logical_and(np.logical_and(X_train_sum, np.logical_and(X_test_sum, X_val_sum)),T_sum)
X_cat_train = X_cat_train.loc[:, index]
X_cat_test = X_cat_test.loc[:, index]
X_cat_val = X_cat_val.loc[:, index]
T_cat = T_cat.loc[:,index]
X_cat_test.head()

,gender,source_system_tab|discover,source_system_tab|explore,source_system_tab|listen with,source_system_tab|my library,source_system_tab|notification,source_system_tab|radio,source_system_tab|search,source_system_tab|settings,source_type|album,...,city|13,city|14,city|15,city|16,city|17,city|18,city|19,city|20,city|21,city|22
5296846,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5530281,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5768063,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
174007,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3260021,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
def Matrix_mult(*args):
	"""An internal method to multiply matrices."""
	return functools.reduce(np.dot, args)


# MCA

In [11]:
X_train = X_cat_train
size = [45000, X_train.shape[1]]
N_all = np.sum(X_train.values[0:size[0],:])

Z_residual = X_train.values[0:size[0],:] / N_all
Z_residual.shape

(50000, 42)

In [12]:
Sum_r = np.sum(Z_residual, axis=1)
Sum_c = np.sum(Z_residual, axis=0)
print(size, Sum_r.shape, Sum_c.shape, N_all)

[50000, 42] (50000,) (42,) 176410


In [13]:
Z_expected = np.outer(Sum_r, Sum_c)
Z_residual -= Z_expected
del Z_expected
gc.collect()

272

In [14]:
D_r_sqrt_mi = np.sqrt(np.diag(Sum_r ** -1))
del Sum_r
gc.collect()
D_c_sqrt_mi = np.sqrt(np.diag(Sum_c ** -1))
del Sum_c
gc.collect()

0

In [28]:
MCA_mat = Matrix_mult(D_r_sqrt_mi, Z_residual, D_c_sqrt_mi)

In [29]:
P, S, Q = np.linalg.svd(MCA_mat)

In [30]:
S_d = diagsvd(S, size[0], size[1])

In [31]:
F = Matrix_mult(D_r_sqrt_mi, P, S_d)  ## Column Space, contains linear combinations of columns
G = Matrix_mult(D_c_sqrt_mi, Q.T, S_d.T)  ## Row space, contains linear combinations of rows

In [33]:
del  Q, P, MCA_mat, S_d, D_r_sqrt_mi, D_c_sqrt_mi
gc.collect()

288

In [35]:
Lam = S ** 2
Expl_var = Lam / np.sum(Lam)

print('Eigen values are ', Lam)
print('Explained variance of eigen vectors are ', Expl_var)

data = {'Iλ': pd.Series(Lam),
		'τI': pd.Series(Expl_var)}
columns = ['Iλ', 'τI']#, 'Zλ', 'τZ', 'cλ', 'τc']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'
np.round(table2.astype(float), 4)
table2.to_csv('table2.csv')

Eigen values are  [  5.75198812e-01   5.52087819e-01   5.46940114e-01   4.51365186e-01
   3.88510834e-01   3.14711006e-01   3.11861901e-01   3.09348475e-01
   3.08178081e-01   2.99110340e-01   2.97215220e-01   2.95808155e-01
   2.94130357e-01   2.92918909e-01   2.91575603e-01   2.90477488e-01
   2.89447530e-01   2.88868070e-01   2.88321983e-01   2.87695989e-01
   2.87218273e-01   2.86732746e-01   2.86445510e-01   2.84890041e-01
   2.84331454e-01   2.83799272e-01   2.81717072e-01   2.81236870e-01
   2.79506298e-01   2.78053541e-01   2.75158345e-01   2.73355157e-01
   2.65008369e-01   2.07277325e-01   1.28693984e-01   1.16478729e-01
   3.55503874e-02   2.57806426e-02   7.36115649e-03   2.72612379e-28
   1.10252561e-28   1.88220873e-29]
Explained variance of eigen vectors are  [  5.16226767e-02   4.95485220e-02   4.90865282e-02   4.05089138e-02
   3.48678905e-02   2.82445377e-02   2.79888376e-02   2.77632638e-02
   2.76582237e-02   2.68444164e-02   2.66743340e-02   2.65480533e-02
   2.639

In [39]:
columns_keep = 30
print('Varianza explicada tomando '+str(columns_keep)+' features: ', table2['τI'][0:columns_keep].sum())

Varianza explicada tomando 30 features:  0.88021718463


In [63]:
X_cat_trainMatrix= Matrix_mult(X_train.values, G[:, :columns_keep]) / S[:columns_keep] / 10
X_cat_testMatrix = Matrix_mult(X_cat_test.values, G[:, :columns_keep]) / S[:columns_keep] / 10
X_cat_valMatrix = Matrix_mult(X_cat_val.values, G[:, :columns_keep]) / S[:columns_keep] / 10
T_catMatrix = Matrix_mult(T_cat.values, G[:, :columns_keep]) / S[:columns_keep] / 10

In [64]:
names = ['V' + str(i) for i in range(columns_keep)]
X_cat_train = pd.DataFrame(X_cat_trainMatrix, columns=names, index = X_cat_train.index)
X_cat_test = pd.DataFrame(X_cat_testMatrix, columns=names, index =X_cat_test.index)
X_cat_val = pd.DataFrame(X_cat_valMatrix, columns=names, index =X_cat_val.index)  
T_cat = pd.DataFrame(T_catMatrix, columns=names, index =T_cat.index)



In [65]:
X_cat_train.head()


,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29
3584192,0.045205,0.007452,-0.308407,-0.170956,0.046271,0.070876,0.143434,-0.135219,-0.005512,-0.021050,...,-0.020771,-0.072485,-0.008289,0.021705,-0.016588,0.059944,0.015366,0.033528,-0.194030,0.089883
2303057,0.063932,-1.559632,0.297920,-0.073250,-0.010165,0.111453,0.030921,0.126575,-0.024555,0.250103,...,-0.143324,-0.230448,-0.765025,0.254882,0.224112,-0.025451,0.204761,-0.013018,-0.083080,0.042256
3163611,0.071165,-0.006337,-0.166710,0.732966,0.038854,-0.183980,-0.005017,-0.281268,-0.179459,0.137904,...,0.022049,-0.046964,-0.006040,0.144254,-0.047696,0.250177,0.035408,-0.093263,0.165276,0.000810
4019357,0.065592,-0.012485,-0.319091,-0.182700,0.044248,-0.093930,-0.048530,-0.110607,0.104395,0.112649,...,0.031688,0.022658,0.004538,-0.023184,0.022805,-0.045119,-0.157027,-0.036335,-0.008509,0.049617
519966,0.094386,0.072793,0.282492,-0.058865,0.021940,-0.064583,0.162020,-0.337542,-0.063068,0.038656,...,0.097736,-0.136229,0.014043,0.073232,0.033035,0.112915,0.128634,0.030883,0.007358,0.001656


In [66]:
T_cat.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29
0,0.076180,0.108900,0.214637,-0.036881,0.008579,0.063576,-0.076016,0.271113,-0.018306,-0.030078,...,0.083740,0.068472,-0.009434,-0.075019,-0.025948,0.036152,0.007881,0.021854,-0.012315,-0.019384
1,0.076180,0.108900,0.214637,-0.036881,0.008579,0.063576,-0.076016,0.271113,-0.018306,-0.030078,...,0.083740,0.068472,-0.009434,-0.075019,-0.025948,0.036152,0.007881,0.021854,-0.012315,-0.019384
2,0.061006,0.001803,-0.389486,-0.230595,0.270045,-0.049213,-0.036141,0.113977,-0.037133,-0.604805,...,0.045877,0.161931,0.031488,0.196503,0.152007,-0.211658,0.132512,-0.117608,0.016401,0.113483
3,-0.972572,0.037565,0.082171,-0.071431,0.039667,0.077024,-0.029618,0.235591,-0.059150,0.272271,...,0.109056,-0.145898,0.301990,0.607730,0.357960,-0.072448,0.232007,-0.201790,0.115807,0.015108
4,-0.972572,0.037565,0.082171,-0.071431,0.039667,0.077024,-0.029618,0.235591,-0.059150,0.272271,...,0.109056,-0.145898,0.301990,0.607730,0.357960,-0.072448,0.232007,-0.201790,0.115807,0.015108


# Saving data

In [81]:
concat_Train = pd.concat([X_cat_train, X_num_train], axis=1, ignore_index=False)
concat_Test = pd.concat([X_cat_test, X_num_test], axis=1, ignore_index=False)
concat_Val = pd.concat([X_cat_val, X_num_val], axis=1, ignore_index=False)
concat_T= pd.concat([T_cat, T_num], axis=1, ignore_index=False)

In [86]:
concat_Train = pd.DataFrame(stdc().fit_transform(concat_Train.values), columns=concat_Train.columns, index = concat_Train.index)
concat_Test = pd.DataFrame(stdc().fit_transform(concat_Test.values), columns=concat_Test.columns, index = concat_Test.index)
concat_Val = pd.DataFrame(stdc().fit_transform(concat_Val.values), columns=concat_Val.columns, index = concat_Val.index)
concat_T = pd.DataFrame(stdc().fit_transform(concat_T.values), columns=concat_T.columns, index = concat_T.index)

In [89]:
concat_Train.isnull().sum().sum()

0

In [88]:
concat_Train.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,V29,song_length,song_year,target
3584192,0.174063,0.027020,-1.208726,-0.733442,0.233784,0.294051,0.628941,-0.660707,-0.028454,-0.122708,...,0.109521,-0.063212,0.307080,0.094488,0.196336,-0.982203,0.494823,-0.393152,0.537425,0
2303057,0.244532,-6.009164,1.146019,-0.316344,-0.053983,0.472668,0.143460,0.615787,-0.140553,1.350079,...,1.315154,1.023448,-0.149204,1.131160,-0.065307,-0.418099,0.231357,-0.032418,0.227031,0
3163611,0.271749,-0.026091,-0.658428,3.125341,0.195964,-0.827799,-0.011608,-1.372840,-1.052424,0.740662,...,0.743154,-0.203653,1.323541,0.204192,-0.516385,0.844629,0.002092,0.469210,-0.393756,1
4019357,0.250781,-0.049773,-1.250222,-0.783576,0.223467,-0.431410,-0.199363,-0.540699,0.618536,0.603484,...,-0.122579,0.114631,-0.254294,-0.849115,-0.196378,-0.038954,0.272081,-0.664094,0.692622,1
519966,0.359130,0.278704,1.086101,-0.254932,0.109718,-0.302226,0.709140,-1.647227,-0.367268,0.201585,...,0.375935,0.160813,0.590115,0.714469,0.181469,0.041720,0.006770,-0.382306,0.692622,1


In [87]:
concat_Train['target'] = y_train
concat_Test['target']= y_test
concat_Val['target']= y_val


In [90]:
concat_T.to_csv('../Data/preprocessedOriginalTMCA.csv')
concat_Train.to_csv('../Data/preprocessedTrainMCA.csv')
concat_Test.to_csv('../Data/preprocessedTestMCA.csv')
concat_Val.to_csv('../Data/preprocessedValMCA.csv')